In [1]:
import pandas as pd
import os

# Define the fixed ladder bitrates
ladder_bitrates = [145, 300, 600, 900, 1600, 2400, 3400, 4500, 5800, 8100, 11600, 16800]

# Define the resolutions to consider
resolutions = [360, 540, 720, 1080, 1440, 2160]

# Define the folder where the CSV files are located
csv_folder_path = "C:\\Angeliki\\vvenc-faster\\vvenc_consolidated_faster\\"

# Generate the ladder for each video
video_ladders = []

for video_num in range(1, 1001):
    video_name = f"test{video_num:04d}"
    
    # Determine which set the video belongs to
    if video_num >= 1 and video_num <= 250:
        set_number = 1
    elif video_num >= 251 and video_num <= 500:
        set_number = 2
    elif video_num >= 501 and video_num <= 750:
        set_number = 3
    else:    
        set_number = 4

    # Load all resolution CSVs for the video
    video_data = []
    for resolution in resolutions:
        resolution_str = f"{resolution}p"
        csv_filename = f"output_{resolution_str}_Set{set_number}.csv"
        csv_filepath = os.path.join(csv_folder_path, csv_filename)
        
        if os.path.exists(csv_filepath):
            df = pd.read_csv(csv_filepath)
            df['resolution'] = resolution  # Add the resolution as a new column
            
            if resolution == 720 and set_number == 1:
                specific_video_name = f"{video_num:04d}"
            else:
                specific_video_name = video_name

            df_filtered = df[df['video_name'] == specific_video_name]
            video_data.append(df_filtered)
    
    # Concatenate all resolutions data for the video
    if video_data:
        video_df = pd.concat(video_data)
    else:
        continue

    # Generate the ladder for the video
    for idx, target_bitrate in enumerate(ladder_bitrates):
        best_resolution = None
        best_vmaf = float('-inf')
        selected_row = None

        # Filter to exclude bitrates >= next target bitrate
        if idx < len(ladder_bitrates) - 1:
            next_target_bitrate = ladder_bitrates[idx + 1]
            filtered_df = video_df[video_df['bitrate'] < next_target_bitrate]
        else:
            filtered_df = video_df[video_df['bitrate'] <= 1.25 * target_bitrate]

        if not filtered_df.empty:
            # Find the bitrate closest to the target bitrate
            filtered_df['bitrate_diff'] = (filtered_df['bitrate'] - target_bitrate).abs()
            closest_rows = filtered_df.loc[filtered_df.groupby('resolution')['bitrate_diff'].idxmin()]

            for _, row in closest_rows.iterrows():
                # Calculate VMAF measure
                vmaf_measure = row['VMAF']

                # Check if this is the best XPSNR measure found so far
                if vmaf_measure > best_vmaf:
                    best_vmaf = vmaf_measure
                    best_resolution = row['resolution']
                    selected_row = row

        # Add the best resolution for this target bitrate to the ladder
        if selected_row is not None:
            video_ladder = {
                "video_name": video_name,
                "target_bitrate": target_bitrate,
                "resolution": best_resolution,
                "selected_bitrate": selected_row['bitrate'],
                "PSNRY": selected_row['PSNRY'],
                "PSNRU": selected_row['PSNRU'],
                "PSNRV": selected_row['PSNRV'],
                "PSNR" : (6 * selected_row['PSNRY'] + selected_row['PSNRU'] + selected_row['PSNRV']) / 8,
                "XPSNRY": selected_row['XPSNRY'],
                "XPSNRU": selected_row['XPSNRU'],
                "XPSNRV": selected_row['XPSNRV'],
                "XPSNR" : (6 * selected_row['XPSNRY'] + selected_row['XPSNRU'] + selected_row['XPSNRV']) / 8,
                "VMAF": selected_row['VMAF'],
                "enc_time": selected_row['enc_time'],
                "dec_time": selected_row['dec_time']
            }
            video_ladders.append(video_ladder)

# Convert the results to a DataFrame and save to a new CSV file
ladder_df = pd.DataFrame(video_ladders)
ladder_df.to_csv("C:\\Angeliki\\vvenc-faster\\fixed_bitrate_dynres_ladder_vmaf.csv", index=False)

# Print the DataFrame for debugging
print(ladder_df)


C:\Users\balakrishnamenon\AppData\Local\Temp\ipykernel_20100\3897966337.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['bitrate_diff'] = (filtered_df['bitrate'] - target_bitrate).abs()


NameError: name 'best_vmaf' is not defined

In [4]:
# Convert the results to a DataFrame and save to a new CSV file
ladder_df = pd.DataFrame(video_ladders)
ladder_df.to_csv("C:\\Angeliki\\vvenc-faster\\fixed_bitrate_dynres_ladder_xpsnr.csv", index=False)